In [ ]:
pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install -y python3-tk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.8.10-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [ ]:
import os
import multiprocessing
import threading
from Crypto.Cipher import AES
from Crypto.Util import Counter
import time
import numpy as np
import math
import random
import string
import secrets
import ctypes
import sys
import itertools
from mpi4py import MPI

In [ ]:
# MPI initialization
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [ ]:
# AES key and block size
key_size = 16
block_size = 16

In [ ]:
# Function for padding the input data
def pad_data(data):
    padding = block_size - len(data) % block_size
    return data + bytes([padding] * padding)

In [ ]:
# Function for unpadding the decrypted data
def unpad_data(data):
    padding = data[-1]
    return data[:-padding]

In [ ]:
# Function for encrypting data using AES algorithm
def encrypt_data(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_data = pad_data(data)
    encrypted_data = cipher.encrypt(padded_data)
    return encrypted_data

In [ ]:
# Function for decrypting data using AES algorithm
def decrypt_data(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_data = cipher.decrypt(data)
    unpadded_data = unpad_data(decrypted_data)
    return unpadded_data

In [ ]:
# Main function for encryption/decryption
def main():
    # Get input for encryption/decryption
    if rank == 0:
        option = input("Select option:\n1. Encryption\n2. Decryption\nEnter option number: ")
        if option == "1":
            plaintext = input("Enter plaintext: ")
            data = plaintext.encode()
        elif option == "2":
            ciphertext = input("Enter ciphertext: ")
            data = bytes.fromhex(ciphertext)
        else:
            print("Invalid option!")
            return
    else:
        data = None
    # Broadcast input data to all processes
    data = comm.bcast(data, root=0)
    # Generate AES key
    if rank == 0:
        key = input("Enter AES key ({} bytes): ".format(key_size)).encode()
        key = key[:key_size]
    else:
        key = None
    # Broadcast AES key to all processes
    key = comm.bcast(key, root=0)
    # Encrypt or decrypt data using AES algorithm
    start_time = time.time()
    if option == "1":
        chunk_size = len(data) // size
        sendbuf = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
        recvbuf = comm.scatter(sendbuf, root=0)
        encrypted_data = encrypt_data(recvbuf, key)
        encrypted_data = comm.gather(encrypted_data, root=0)
        if rank == 0:
            ciphertext = b"".join(encrypted_data).hex()
            print("Ciphertext: {}".format(ciphertext))
    elif option == "2":
        chunk_size = len(data) // size
        sendbuf = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
        recvbuf = comm.scatter(sendbuf, root=0)
        decrypted_data = decrypt_data(recvbuf, key)
        decrypted_data = comm.gather(decrypted_data, root=0)
        if rank == 0:
            plaintext = b"".join(decrypted_data).decode()
            print("Plaintext: {}".format(plaintext))
    end_time = time.time()
    # Display execution time
    if rank == 0:
        print("Execution time: {:.6f} seconds".format(end_time - start_time))

if __name__ == "__main__":
    main()

Select option:
1. Encryption
2. Decryption
Enter option number: 1
Enter plaintext: Password: Rushil123
Enter AES key (16 bytes): 0x2b7e151628aed2a6abf7158809cf4f3
Ciphertext: 6b235c03fec30609c6b03dde06f58895c5cf797458d6e26c6a5fd6bd69ffa1e8
Execution time: 0.000265 seconds
